In [1]:
from llm_provider import get_llm
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [2]:
def stream_res(response):
    for token in response:
        print(token, end="", flush=True)

In [6]:
from langchain_core.prompts import load_prompt

prompt = load_prompt("prompts/capital.yaml", encoding="utf-8")
print(prompt.format(country="대한민국"))

대한민국의 수도에 대해서 알려주세요.
수도의 특징을 다음의 양식에 맞게 정리해 주세요.
300자 내외로 작성해 주세요.
한글로 작성해 주세요.
----
[양식]
1. 면적
2. 인구
3. 역사적 장소
4. 특산품

#Answer:



In [7]:
llm = get_llm(temperature=0.1)
# llm = get_llm(provider="ollama",model="llama3.2")
chain = prompt|llm|StrOutputParser()

In [8]:
# input = {"topic": "김치"}
# response = chain.invoke(input)

response = chain.invoke("대한민국")
print(response)
# print(response.content)

1. 면적: 서울특별시는 약 605.21㎢로, 대한민국의 수도이자 가장 큰 도시입니다.  
2. 인구: 2023년 기준으로 서울의 인구는 약 9백만 명에 달하며, 국내에서 가장 인구 밀집도가 높은 지역입니다.  
3. 역사적 장소: 경복궁, 창덕궁, 남산 N서울타워 등 다양한 역사적 장소가 있어 한국의 전통과 문화를 체험할 수 있습니다.  
4. 특산품: 서울의 특산품으로는 한방차, 전통주, 그리고 다양한 길거리 음식들이 있으며, 특히 떡볶이와 순대가 유명합니다.  

서울은 현대와 전통이 조화를 이루는 도시로, 다양한 문화와 역사를 경험할 수 있는 매력적인 장소입니다.


In [7]:
# input = {"topic": "김치"}
# response = chain.stream(input)
input = "대한민국"
response = chain.invoke(input)
stream_res(response)

대한민국의 수도는 Seoul입니다.

 Seoul에 대해서 알려드릴 몇 가지 특징을 다음은 तरह으로 정리했습니다:

1. 면적: 서울은 약 605.49 square kilometres(233.7 平方미ล)으로 크기가 커지며, 이중에 80%가 도로와 공원이ครอบ giữ되어 있습니다.
2. 인구: 서울은 약 10,277,037 명으로 인구가 đôngdense한 도시 중 한 곳입니다.
3. 역사적 장소: 서울은 한국의 역사적인 중심地로, 고대 세종대왕 시대부터 현재까지 많은 역사가와 문화를 보유하고 있습니다. 구세ون시(구 Seoul), 신세원시(신 Seoul), 전남성리와 같은 역사적인 지역이 존재합니다.
4. 특산품: 서울은 한적한 자연을 바탕으로 다양한 식민물과 식물, 수식물과 화분을 보유하고 있으며, 유명한 한국식酒인 마스주(Makgeolli)와 소가고(소고기), 한라산고(韓良山고) 등이 있습니다.

`ChatPromptTemplate` 은 대화목록을 프롬프트로 주입하고자 할 때 활용

메시지는 튜플(tuple) 형식으로 구성하며, (`role`, `message`) 로 구성하여 리스트로 생성할 수 있다

**role**
- `"system"`: 시스템 설정 메시지 이다. 주로 전역설정과 관련된 프롬프트이다
- `"human"` : 사용자 입력 메시지 이다
- `"ai"`: AI 의 답변 메시지이다.

In [10]:
from langchain_core.prompts import ChatPromptTemplate

template = ChatPromptTemplate([
    ("system", "You are a helpful AI bot. Your name is {name}."),
    ("human", "Hello, how are you doing?"),
    ("ai", "I'm doing well, thanks!"),
    ("human", "{user_input}"),
])

prompt_value = template.invoke(
    {
        "name": "Bob",
        "user_input": "What is your name?"
    }
)

prompt_value

ChatPromptValue(messages=[SystemMessage(content='You are a helpful AI bot. Your name is Bob.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello, how are you doing?', additional_kwargs={}, response_metadata={}), AIMessage(content="I'm doing well, thanks!", additional_kwargs={}, response_metadata={}), HumanMessage(content='What is your name?', additional_kwargs={}, response_metadata={})])

In [11]:
response = llm.invoke(prompt_value)
response.content

'My name is Bob. How can I assist you today?'

## FewShotPromptTemplate


In [12]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

# 예제 데이터 정의
examples = [
    {
        "question": "아이유로 삼행시 만들어줘",
        "answer": """
아: 아이유는
이: 이런 강의를 들을 이
유: 유가 없다.
"""
    },
    {
        "question": "김민수로 삼행시 만들어줘",
        "answer": """
김: 김치는 맛있다
민: 민달팽이도 좋아하는 김치!
수: 수억을 줘도 김치는 내꺼!
"""
    }
]

# 예제 포맷 템플릿 정의
example_prompt = PromptTemplate(
    input_variables=["question", "answer"],
    template="Question: {question}\n{answer}"
)

# FewShotPromptTemplate 생성
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,  # 사용할 예제들
    example_prompt=example_prompt,  # 예제 포맷 템플릿
    suffix="Question:\n{question}\nAnswer:", # 사용자 입력에 추가될 접미사
    input_variables=["question"]  # 입력 변수 정의
)

In [13]:
# 사용자 질문 정의
question = "장보고  삼행시 만들어줘"

chain = few_shot_prompt | llm | StrOutputParser()
# 최종 프롬프트 생성 및 출력
answer = chain.stream(
    {"question": question}
)
stream_res(answer)

장: 장대한 꿈을 가진
보: 보물 같은 사람,
고: 고난을 이겨내며 나아간다.

In [14]:

prompt_template = """You are a helpful expert in summary writing. You are given lists of summaries.
Please sum up previously summarized sentences according to the following REQUEST.
REQUEST:
1. Summarize the main points in bullet points in KOREAN.
2. Each summarized sentence must start with an emoji that fits the meaning of the each sentence.
3. Use various emojis to make the summary more interesting.
4. MOST IMPORTANT points should be organized at the top of the list.
5. DO NOT include any unnecessary information.

LIST OF SUMMARIES:
{doc_summaries}

Helpful Answer: """
prompt = ChatPromptTemplate.from_template(prompt_template)

prompt

ChatPromptTemplate(input_variables=['doc_summaries'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['doc_summaries'], input_types={}, partial_variables={}, template='You are a helpful expert in summary writing. You are given lists of summaries.\nPlease sum up previously summarized sentences according to the following REQUEST.\nREQUEST:\n1. Summarize the main points in bullet points in KOREAN.\n2. Each summarized sentence must start with an emoji that fits the meaning of the each sentence.\n3. Use various emojis to make the summary more interesting.\n4. MOST IMPORTANT points should be organized at the top of the list.\n5. DO NOT include any unnecessary information.\n\nLIST OF SUMMARIES:\n{doc_summaries}\n\nHelpful Answer: '), additional_kwargs={})])

In [17]:
# 사용자 질문 정의
question = "장보고  삼행시 만들어줘"

chain = few_shot_prompt | llm | StrOutputParser()
# 최종 프롬프트 생성 및 출력
answer = chain.stream(
    {"question": question}
)
stream_res(answer)

장: 장대한 꿈을 가진
보: 보물 같은 인생을
고: 고대하는 사람!